In [1]:
# First principles: a decorator is a callable that takes a function (or class)
# and returns a new function (or class).

def log_calls(fn):
    # 'fn' is the function being decorated
    def wrapper(*args, **kwargs):
        print(f"→ Calling {fn.__name__} with args={args}, kwargs={kwargs}")
        result = fn(*args, **kwargs)
        print(f"← {fn.__name__} returned {result!r}")
        return result
    return wrapper  # Returning the new callable

@log_calls  # Equivalent to: add = log_calls(add)
def add(a, b):
    """Add two numbers."""
    return a + b

@log_calls
def greet(name):
    """Return a greeting."""
    return f"Hello, {name}!"

# Using the decorated functions
sum_result = add(3, 4)
message = greet("dkp")

→ Calling add with args=(3, 4), kwargs={}
← add returned 7
→ Calling greet with args=('dkp',), kwargs={}
← greet returned 'Hello, dkp!'


In [2]:
from functools import wraps

def log_calls(fn):
    @wraps(fn)  # Preserves fn.__name__, fn.__doc__, etc.
    def wrapper(*args, **kwargs):
        print(f"→ {fn.__name__}({args}, {kwargs})")
        result = fn(*args, **kwargs)
        print(f"← {result!r}")
        return result
    return wrapper

@log_calls
def multiply(a: int, b: int) -> int:
    """Multiply two integers together."""
    return a * b

# Using the decorated function
product = multiply(6, 7)

# Checking the preserved metadata
print(f"Name: {multiply.__name__}")
print(f"Doc: {multiply.__doc__}")
print(f"Annotations: {multiply.__annotations__}")

→ multiply((6, 7), {})
← 42
Name: multiply
Doc: Multiply two integers together.
Annotations: {'a': <class 'int'>, 'b': <class 'int'>, 'return': <class 'int'>}


In [3]:
from functools import wraps
import time
import random

def retry(tries=3):
    """A decorator that retries the function call if it fails."""
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            for i in range(tries):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {i+1} failed: {e}")
                    if i == tries - 1:
                        print("No more retries — raising exception.")
                        raise
                    wait = 2**i * 0.1
                    print(f"Retrying in {wait:.1f} seconds...")
                    time.sleep(wait)
        return wrapper
    return deco

@retry(tries=5)  # Customize retry attempts
def flaky_function():
    """Randomly fails to simulate instability."""
    if random.random() < 0.7:  # 70% chance to fail
        raise ValueError("Random failure occurred.")
    return "Success!"

# Running the decorated function
print(flaky_function())

Attempt 1 failed: Random failure occurred.
Retrying in 0.1 seconds...
Attempt 2 failed: Random failure occurred.
Retrying in 0.2 seconds...
Success!


In [4]:
from dataclasses import dataclass

# Class decorator that injects a __repr__ method
def add_repr(cls):
    def __repr__(self):
        return f"{cls.__name__}({self.__dict__})"
    cls.__repr__ = __repr__
    return cls

@add_repr
class C:
    def __init__(self, x):
        self.x = x

@dataclass
@add_repr
class Point:
    x: int
    y: int

# Using the decorated classes
c = C(42)
p = Point(3, 4)

print(c)  # Output: C({'x': 42})
print(p)  # Output: Point({'x': 3, 'y': 4})

C({'x': 42})
Point({'x': 3, 'y': 4})


In [5]:
import time

class cached_property:
    """A data descriptor that caches the result of a method after first call."""
    def __init__(self, fn):
        self.fn = fn
        self.attr = f"_{fn.__name__}_cache"

    def __get__(self, obj, owner):
        if obj is None:  # Accessed on the class, not instance
            return self
        if not hasattr(obj, self.attr):
            print(f"Computing {self.fn.__name__} for the first time...")
            setattr(obj, self.attr, self.fn(obj))
        return getattr(obj, self.attr)

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @cached_property
    def area(self):
        """Expensive area computation (simulated with sleep)."""
        time.sleep(1)  # Simulate slow calculation
        return 3.14159 * (self.radius ** 2)

# --- Using the cached property ---
c = Circle(10)

print("First access to c.area:")
print(c.area)  # Computes and caches
print("\nSecond access to c.area:")
print(c.area)  # Returns cached value instantly

First access to c.area:
Computing area for the first time...
314.159

Second access to c.area:
314.159


### Pitfalls & Testing

1. **Hidden side-effects**

   * Without a clear docstring or using `functools.wraps` for function decorators, the fact that something is cached might be unclear to maintainers.
   * This can cause bugs if the underlying data changes but the cached result doesn’t update.

2. **Single responsibility**

   * Keep decorators focused — this `cached_property` only does caching.
   * Don’t also add logging, retries, or unrelated features; that makes it harder to reason about and test.

3. **Testing**

   * In unit tests, explicitly check that the function is called only once per instance.
   * You might also add a `.clear_cache()` method to the decorator for testing and debugging.

## Exercises
1. Implement `timeit` decorator that records run time to a logger.
2. Write an `authorize(role)` decorator that checks a context object.
3. Build `validate(schema)` that validates kwargs and raises `TypeError`.